In [30]:
import glob

In [34]:
import numpy as np
import pandas as pd

In [33]:
import bokeh.plotting
import bokeh.layouts
from bokeh.palettes import Category10_10 as palette

# Housing Affordability Data System
Data provided by:<br />
Office of Policy Development and Research (PD&R)<br />
U.S. Department of Housing and Urban Development<br />
Secretary Ben Carson<br />
https://www.huduser.gov/portal/datasets/hads/hads.html

## Load the data

In [8]:
! ls -lh data/

total 651M
-rw-rw-r-- 1 schowell schowell  37M Feb 28  2007 hads1985.txt
-rw-rw-r-- 1 schowell schowell  37M Feb 28  2007 hads1987.txt
-rw-rw-r-- 1 schowell schowell  39M Feb 28  2007 hads1989.txt
-rw-rw-r-- 1 schowell schowell  38M Feb 28  2007 hads1991.txt
-rw-rw-r-- 1 schowell schowell  41M Oct  3  2007 hads1993.txt
-rw-rw-r-- 1 schowell schowell  38M Feb 28  2007 hads1995.txt
-rw-rw-r-- 1 schowell schowell  36M Feb 28  2007 hads1997.txt
-rw-rw-r-- 1 schowell schowell  40M Feb 28  2007 hads1999.txt
-rw-rw-r-- 1 schowell schowell  36M Feb 28  2007 hads2001.txt
-rw-rw-r-- 1 schowell schowell  41M Feb 28  2007 hads2003.txt
-rw-rw-r-- 1 schowell schowell  37M Jan 19  2011 hads2005.txt
-rw-rw-r-- 1 schowell schowell  34M Jan 19  2011 hads2007.txt
-rw-rw-r-- 1 schowell schowell  39M Jun 25  2010 hads2009.txt
-rw-r----- 1 schowell schowell 115M Apr 12  2013 hads2011.txt
-rw-r----- 1 schowell schowell  51M Jun 24  2015 hads2013.txt


The entire data set is 651M, let only grab some of these files.

In [35]:
fnames = glob.glob('data/*txt')
fnames.sort()

In [36]:
fnames

['data/hads1985.txt',
 'data/hads1987.txt',
 'data/hads1989.txt',
 'data/hads1991.txt',
 'data/hads1993.txt',
 'data/hads1995.txt',
 'data/hads1997.txt',
 'data/hads1999.txt',
 'data/hads2001.txt',
 'data/hads2003.txt',
 'data/hads2005.txt',
 'data/hads2007.txt',
 'data/hads2009.txt',
 'data/hads2011.txt',
 'data/hads2013.txt']

In [28]:
years = [int(fname.strip('data/hads.txt')) for fname in fnames]

In [29]:
years

[1985,
 1987,
 1989,
 1991,
 1993,
 1995,
 1997,
 1999,
 2001,
 2003,
 2005,
 2007,
 2009,
 2011,
 2013]

In [15]:
f = fnames[0]

In [26]:
f.strip('abcdefghijklmnopqrstuvwxyz/.')

'1985'

In [18]:
f.strip('data/hads')

'1985.tx'